MPTS: Multi-day Pairs Trading Strategy

In [1]:
import pandas as pd
import numpy as np
import all_functions as af
from tqdm import tqdm

Import data. Best Pairs is created from pairs_building.ipynb

In [2]:
all_pairs = pd.read_excel('best_pairs.xlsx')

In [3]:
all_pairs.head()

,test_end,eg1,eg2,trace0,trace1,eig0,eig1,w1,w2,joh_sig,eg,eg_sig,coint,y,x
0,2016-12-31,0.004707,0.000784,25.547552,7.839195,17.708357,7.839195,3.809466,-4.410000,True,0.000784,True,True,XEL,CMS
1,2016-12-31,0.007685,0.232423,21.580337,5.351801,16.228535,5.351801,0.627657,-0.105774,True,0.007685,True,True,AME,HON
2,2016-12-31,0.012235,0.624439,23.143461,5.520720,17.622741,5.520720,0.634995,-0.016151,True,0.012235,True,True,AME,ITW
3,2016-12-31,0.013444,0.029365,24.340426,7.513910,16.826516,7.513910,0.732184,-0.532618,True,0.013444,True,True,APH,TEL
4,2016-12-31,0.104706,0.010474,20.316993,5.055470,15.261524,5.055470,0.160964,-0.410497,True,0.010474,True,True,WMT,COST


In [4]:
data = pd.read_excel('price_matrix.xlsx')
data.index = data['Date']
data.drop('Date', axis = 1, inplace = True)
data = data['2016-12-31':]

In [5]:
#Take only best pairs data
best_pairs = pd.unique(all_pairs[['y','x']].values.ravel('K'))
data = data[best_pairs]

In [6]:
data.head()

,XEL,AME,APH,WMT,DISCK,CMS,HON,ITW,TEL,COST,DISCA
Date,,,,,,,,,,,
2017-01-05,0.000000,-0.012648,-0.005300,0.002172,-0.037278,0.005508,0.002231,-0.011220,-0.012087,0.019717,-0.038530
2017-01-06,0.002941,0.018097,0.007548,-0.013726,0.009774,0.003810,0.015160,0.010443,0.006781,-0.000491,0.010294
2017-01-09,-0.015152,-0.008788,-0.010282,0.006592,-0.022710,-0.013523,-0.008268,-0.004883,-0.014495,-0.011423,-0.022198
2017-01-10,-0.000248,0.004836,0.006085,-0.006986,-0.018667,-0.001684,0.002552,0.002372,0.004308,0.004287,-0.019725
2017-01-11,0.009928,0.002206,0.000442,0.004397,0.008929,0.010600,0.001358,0.004813,0.013610,-0.000742,0.006454


In [7]:
zscore_open = 1.5 #Open at 1.5
zscore_close = 0.5 #Close at 0.5

In [8]:
#Create an empty dataframe for log_book
log_book = pd.DataFrame(columns = ['Open_date','Close_date','Pair_Name','Open_price_Y','Close_price_Y','Shares_Y','Open_price_X', \
                                   'Close_price_X', 'Shares_X', 'Signal', 'Returns'])
log_index = 0

Backtest loop for every pair

In [10]:
for i in tqdm(range(len(all_pairs))):
    
    pair = all_pairs.loc[i][['y','x']]
   
    
    #For some stocks there neither BBG code is available nor DSCD
    
    stockY = pair['y']
    
    stockX = pair['x']
        
    prices = data[pair].apply(af.KFSmoother)
    
    prices['hedge_ratio'] = af.KFHedgeRatio(y = prices[pair['y']], x = prices[pair['x']])[:,0]
    prices['spread'] = prices[pair['y']].add(prices[pair['x']].mul(prices.hedge_ratio))
    half_life = af.estimate_half_life(prices.spread)
    max_window = len(prices.index)
    spread = prices.spread.rolling(window=min(2 * half_life, max_window))
    prices['z_score'] = prices.spread.sub(spread.mean()).div(spread.std())

    #Signal Generation
    prices['Long'] = (prices['z_score'] < -zscore_open)
    prices['Short'] = (prices['z_score'] > zscore_open)
    
    #Dummy variables
    buy_open = 0
    sell_open = 0
    posTaken = 0
    
    #For each pair, go inside this loop and generate daywise signals
    for j in range(len(prices)):
        
        #Closing a position loop starts
        if posTaken:    #If position is taken, then we will be able to close it
            
            if buy_open and (prices['z_score'][j] > -zscore_close):  #If buy position is there and z_score has breached -0.5 level on the upside, then close buy position
                
                log_book.loc[log_index,'Close_price_Y'] = data[pair['y']][prices.index[j]]
                log_book.loc[log_index,'Close_price_X'] = data[pair['x']][prices.index[j]]
                log_book.loc[log_index,'Close_date'] = prices.index[j]
                
                #Returns for stock Y in Y_X pair
                returnsY = (log_book['Close_price_Y'][log_index]/log_book['Open_price_Y'][log_index]) - 1
                
                #Returns for stock X in Y_X pair
                returnsX = 1 - (log_book['Close_price_X'][log_index]/log_book['Open_price_X'][log_index])
                
                log_book.loc[log_index, 'Returns'] = returnsY * log_book['Shares_Y'][log_index] + returnsX * log_book['Shares_X'][log_index]
                
                #When closed then assign every variable as zero
                posTaken = 0
                buy_open = 0
                sell_open = 0
                log_index += 1
                
            elif sell_open and (prices['z_score'][j] < zscore_close):
                
                log_book.loc[log_index,'Close_price_Y'] = data[pair['y']][prices.index[j]]
                log_book.loc[log_index,'Close_price_X'] = data[pair['x']][prices.index[j]]
                log_book.loc[log_index,'Close_date'] = prices.index[j]
                
                #Returns for stock Y in Y_X pair
                returnsY = 1 - (log_book['Close_price_Y'][log_index]/log_book['Open_price_Y'][log_index])
                
                #Returns for stock X in Y_X pair
                returnsX = (log_book['Close_price_X'][log_index]/log_book['Open_price_X'][log_index]) - 1
                
                log_book.loc[log_index, 'Returns'] = returnsY * log_book['Shares_Y'][log_index] + returnsX * log_book['Shares_X'][log_index]
                
                posTaken = 0
                buy_open = 0
                sell_open = 0
                log_index += 1
        
        #Closing a position loop ends
        
        #Opening position      
        if not posTaken:      
            
            if prices['Long'][j]:
                posTaken = 1
                buy_open = 1
                sell_open = 0
                
                
                log_book.loc[log_index,'Open_date'] = prices.index[j]
                log_book.loc[log_index,'Pair_Name'] =  stockY + '_' + stockX
                log_book.loc[log_index,'Open_price_Y'] = data[pair['y']][prices.index[j]]
                log_book.loc[log_index,'Open_price_X'] = data[pair['x']][prices.index[j]]
                log_book.loc[log_index, 'Signal'] = 'Buy'
                
            elif prices['Short'][j]:
                posTaken = 1
                buy_open = 0
                sell_open = 1
                
                log_book.loc[log_index,'Open_date'] = prices.index[j]
                log_book.loc[log_index,'Pair_Name'] =  stockY + '_' + stockX
                log_book.loc[log_index,'Open_price_Y'] = data[pair['y']][prices.index[j]]
                log_book.loc[log_index,'Open_price_X'] = data[pair['x']][prices.index[j]]
                log_book.loc[log_index, 'Signal'] = 'Sell'
                
            
            #Capital Allocation
            if buy_open or sell_open:
                cap_alloc = 1 #Capital allocation to entire pair is $1
                
                #Formula used here is capital allocation to Y = total_pair_capital/(1+beta) and to X = total_pair_capital * beta /(1+beta)
                #So anytime if there is huge volatility in Y w.r.t X then automatically Y will be given less allocation and vice versa.
                moneyY = cap_alloc/(1+abs(prices['hedge_ratio'][j]))
                moneyX = cap_alloc*abs(prices['hedge_ratio'][j])/(1+abs(prices['hedge_ratio'][j]))
                
                log_book.loc[log_index,'Shares_Y'] = moneyY
                log_book.loc[log_index, 'Shares_X'] = moneyX

100%|██████████| 6/6 [00:05<00:00,  1.13it/s]


In [11]:
log_book.head()

,Open_date,Close_date,Pair_Name,Open_price_Y,Close_price_Y,Shares_Y,Open_price_X,Close_price_X,Shares_X,Signal,Returns
0,2017-01-31 00:00:00,2017-02-01 00:00:00,XEL_CMS,0.014486,-0.016699,0.999922,0.013321,-0.003478,0.000078,Sell,2.152527
1,2017-02-14 00:00:00,2017-02-16 00:00:00,XEL_CMS,-0.007375,0.006729,0.999365,-0.008115,0.007482,0.000635,Buy,-1.909953
2,2017-03-08 00:00:00,2017-03-10 00:00:00,XEL_CMS,-0.017572,0.009984,0.998814,-0.017327,0.009607,0.001186,Buy,-1.564468
3,2017-03-15 00:00:00,2017-03-16 00:00:00,XEL_CMS,0.018497,-0.01067,0.998827,0.017409,-0.013556,0.001173,Sell,1.572893
4,2017-03-29 00:00:00,2017-04-04 00:00:00,XEL_CMS,-0.010238,0.002477,0.998678,-0.007529,0.001785,0.001322,Buy,-1.238687


In [12]:
log_book.to_csv('Log book MPTS.csv', index = None)